In [1]:
"""
=============================================================================
TREINAMENTO BASELINE - DATASET PRÉ-PROCESSADO (SEM AUGMENTATION)
=============================================================================
Pipeline de treinamento para dataset processado:
1. Carrega dataset RAF-DB
2. NÃO aplica augmentation (dados já processados)
3. Treina múltiplos modelos: ResNet50, EfficientNet-B0, EfficientViT
4. Early stopping para evitar overfitting
5. Monitoramento de tempo e memória
6. Salva métricas completas + matriz normalizada + modelo .pth

ESTRATÉGIA BASELINE:
- Dataset: Processado e não balanceado
- Modelos: 3 arquiteturas diferentes
- Métricas: Completas (Acc, F1, Precision, Recall, por classe)
- Monitoramento: Tempo e memória (CPU/GPU)
=============================================================================
"""

import os
import cv2
import gc
import json
import time
import psutil
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    precision_score, 
    recall_score,
    classification_report,
    confusion_matrix,
    precision_recall_fscore_support
)
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')

In [2]:
# =============================================================================
# CONFIGURAÇÕES
# =============================================================================

# Dispositivo
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# Caminhos
DATASET_PATH = r"../data/processed/RAF-DB"
RESULTS_PATH = r"../results/baseline/RAF-DB"
MODELS_PATH = os.path.join(RESULTS_PATH, "models")
METRICS_PATH = os.path.join(RESULTS_PATH, "metrics")
PLOTS_PATH = os.path.join(RESULTS_PATH, "plots")

# Criação de diretórios
os.makedirs(MODELS_PATH, exist_ok=True)
os.makedirs(METRICS_PATH, exist_ok=True)
os.makedirs(PLOTS_PATH, exist_ok=True)

# Parâmetros de Treinamento
EPOCHS = 100
LR = 1e-4
BATCH_SIZE = 64
NUM_WORKERS = 4

# Early Stopping
EARLY_STOP_PATIENCE = 15
EARLY_STOP_MIN_DELTA = 0.001

# Pesos para métricas combinadas
F1_WEIGHT = 0.6
ACC_WEIGHT = 0.4

# Mapeamento de classes
EMOTION_LABELS = {
    'Raiva': 0, 'Nojo': 1, 'Medo': 2, 'Felicidade': 3,
    'Neutro': 4, 'Tristeza': 5, 'Surpresa': 6
}

# Seed para reprodutibilidade
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

print("="*80)
print("TREINAMENTO BASELINE - DATASET PRÉ-PROCESSADO (SEM AUGMENTATION)")
print("="*80)
print(f"Dispositivo: {DEVICE}")
print(f"Dataset: {DATASET_PATH}")
print(f"Épocas máximas: {EPOCHS}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Early Stopping: Patience={EARLY_STOP_PATIENCE}")
print(f"Augmentation: NÃO (dados já processados)")
print("="*80)

TREINAMENTO BASELINE - DATASET PRÉ-PROCESSADO (SEM AUGMENTATION)
Dispositivo: cuda
Dataset: ../data/processed/RAF-DB
Épocas máximas: 100
Batch size: 64
Early Stopping: Patience=15
Augmentation: NÃO (dados já processados)


In [3]:
# CLASSE DE MONITORAMENTO
class ResourceMonitor:
    def __init__(self, model_name):
        self.model_name = model_name
        self.start_time = None
        self.end_time = None
        self.peak_memory_mb = 0
        self.initial_memory_mb = 0
        self.training_time = 0
        self.process = psutil.Process()
        self.peak_gpu_memory_mb = 0
        self.initial_gpu_memory_mb = 0
        self.epoch_times = []

    def record_epoch_time(self, seconds: float):
        # guarda o tempo de época
        self.epoch_times.append(float(seconds))

    def get_epoch_time_stats(self):
        import numpy as np
        if not self.epoch_times:
            return {"mean": None, "std": None, "n": 0}
        arr = np.array(self.epoch_times, dtype=float)
        return {"mean": float(arr.mean()), "std": float(arr.std()), "n": int(arr.size)}
        
    def start_monitoring(self):
        self.start_time = time.time()
        self.initial_memory_mb = self._get_memory_usage()
        self.peak_memory_mb = self.initial_memory_mb
        
        if torch.cuda.is_available():
            self.initial_gpu_memory_mb = torch.cuda.memory_allocated() / 1024**2
            self.peak_gpu_memory_mb = self.initial_gpu_memory_mb
        
        print(f"\n🔍 Iniciando monitoramento: {self.model_name}")
        print(f"  • Initial RAM: {self.initial_memory_mb:.2f} MB")
        if torch.cuda.is_available():
            print(f"  • Initial GPU: {self.initial_gpu_memory_mb:.2f} MB")
        
    def _get_memory_usage(self):
        return self.process.memory_info().rss / 1024 / 1024
        
    def update_peak_memory(self):
        current_memory = self._get_memory_usage()
        if current_memory > self.peak_memory_mb:
            self.peak_memory_mb = current_memory
        
        if torch.cuda.is_available():
            current_gpu = torch.cuda.max_memory_allocated() / 1024**2
            if current_gpu > self.peak_gpu_memory_mb:
                self.peak_gpu_memory_mb = current_gpu
            
    def end_monitoring(self):
        self.end_time = time.time()
        self.training_time = self.end_time - self.start_time
        final_memory_mb = self._get_memory_usage()
        memory_increase = final_memory_mb - self.initial_memory_mb
        
        print("\n" + "="*70)
        print(f"MONITORING REPORT: {self.model_name.upper()}")
        print("="*70)
        print(f"Total training time: {timedelta(seconds=int(self.training_time))}")
        print(f"Initial RAM: {self.initial_memory_mb:.2f} MB")
        print(f"Final RAM: {final_memory_mb:.2f} MB")
        print(f"Peak RAM: {self.peak_memory_mb:.2f} MB")
        print(f"Memory increase: {memory_increase:.2f} MB")
        
        if torch.cuda.is_available():
            final_gpu = torch.cuda.memory_allocated() / 1024**2
            print(f"Peak GPU Memory: {self.peak_gpu_memory_mb:.2f} MB")
            print(f"Final GPU Memory: {final_gpu:.2f} MB")
        
        print("="*70)
        
        return {
            'training_time_seconds': self.training_time,
            'initial_memory_mb': self.initial_memory_mb,
            'final_memory_mb': final_memory_mb,
            'peak_memory_mb': self.peak_memory_mb,
            'memory_increase_mb': memory_increase,
            'peak_gpu_memory_mb': self.peak_gpu_memory_mb if torch.cuda.is_available() else 0
        }

In [4]:
# =============================================================================
# CLASSE DE EARLY STOPPING
# =============================================================================

class EarlyStopping:
    """
    Early Stopping para interromper o treinamento quando a métrica de validação
    não melhora por um número específico de épocas (patience).
    """
    def __init__(self, patience=10, min_delta=0.0, mode='max', verbose=True):
        """
        Args:
            patience (int): Número de épocas sem melhoria antes de parar
            min_delta (float): Mudança mínima para considerar melhoria
            mode (str): 'max' para métricas que devem aumentar (F1, acc),
                       'min' para métricas que devem diminuir (loss)
            verbose (bool): Se True, imprime mensagens
        """
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_score_min = np.inf if mode == 'min' else -np.inf
        
    def __call__(self, val_score):
        """
        Verifica se deve parar o treinamento
        
        Args:
            val_score (float): Métrica de validação atual
            
        Returns:
            bool: True se deve parar o treinamento
        """
        if self.mode == 'max':
            score = val_score
        else:
            score = -val_score
            
        if self.best_score is None:
            self.best_score = score
            self.val_score_min = val_score
            if self.verbose:
                print(f'Early Stopping: Baseline estabelecida: {val_score:.4f}')
        elif score < self.best_score + self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f'Early Stopping: Sem melhoria ({self.counter}/{self.patience})')
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print(f'Early Stopping: Treinamento interrompido após {self.patience} épocas sem melhoria')
        else:
            if self.verbose:
                improvement = val_score - self.val_score_min
                print(f'Early Stopping: Melhoria detectada ({val_score:.4f}, Delta={improvement:+.4f})')
            self.best_score = score
            self.val_score_min = val_score
            self.counter = 0
            
        return self.early_stop

print("Classe Early Stopping configurada")

Classe Early Stopping configurada


In [5]:
# =============================================================================
# DATASET CUSTOMIZADO (SEM AUGMENTATION)
# =============================================================================

class EmotionDataset(Dataset):
    """
    Dataset personalizado para reconhecimento de emoções.
    Apenas normaliza para uso com modelos pré-treinados.
    """
    def __init__(self, root_dir, split='train'):
        self.root_dir = os.path.join(root_dir, split)
        self.image_paths = []
        self.labels = []

        # Carregar todos os caminhos de imagem e rótulos
        for class_name, class_idx in EMOTION_LABELS.items():
            class_path = os.path.join(self.root_dir, class_name)
            if os.path.isdir(class_path):
                for img_name in os.listdir(class_path):
                    if img_name.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                        self.image_paths.append(os.path.join(class_path, img_name))
                        self.labels.append(class_idx)

        if len(self.image_paths) == 0:
            raise ValueError(f"Nenhuma imagem encontrada em {self.root_dir}. Verifique o caminho!")
        
        print(f"\n{split.upper()} Dataset:")
        print(f"   Total de imagens: {len(self.image_paths)}")
        
        # Contar amostras por classe
        label_counts = pd.Series(self.labels).value_counts().sort_index()
        for idx, count in label_counts.items():
            emotion_name = [k for k, v in EMOTION_LABELS.items() if v == idx][0]
            print(f"   {emotion_name:12s}: {count:5d} imagens")

    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        label = self.labels[idx]

        # Ler imagem em escala de cinza (já pré-processada em 224x224)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if image is None:
            raise ValueError(f"Erro ao carregar imagem: {img_path}")
        
        # Normalizar para [0, 1]
        image = image.astype(np.float32) / 255.0
        
        # Normalização com média e desvio padrão (para modelos pré-treinados)
        # Usando normalização simples para grayscale
        image = (image - 0.5) / 0.5
        
        # Adicionar dimensão de canal e converter para tensor
        image = np.expand_dims(image, axis=0)  # [H, W] -> [1, H, W]
        image = torch.from_numpy(image).float()

        return image, torch.tensor(label, dtype=torch.long)

print("Classe Dataset configurada")

Classe Dataset configurada


In [6]:
# =============================================================================
# DEFINIÇÃO DOS MODELOS
# =============================================================================

def create_model(model_name, num_classes=7):
    """
    Cria modelo baseado no nome.
    
    Args:
        model_name: 'resnet50', 'efficientnet_b0', ou 'efficientvit_m5'
        num_classes: Número de classes (7 emoções)
    
    Returns:
        model: Modelo PyTorch
    """
    if model_name == 'resnet50':
        model = models.resnet50(weights='IMAGENET1K_V1')
        
        # Primeira camada adaptada para grayscale (1 canal)
        original_weights = model.conv1.weight.data
        avg_weights = torch.mean(original_weights, dim=1, keepdim=True)
        model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        model.conv1.weight.data = avg_weights
        
        # Camada final para número de classes
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, num_classes)
    
    elif model_name == 'efficientnet_b0':
        model = models.efficientnet_b0(weights='IMAGENET1K_V1')
        
        # Primeira camada adaptada para grayscale (1 canal)
        original_weights = model.features[0][0].weight.data
        avg_weights = torch.mean(original_weights, dim=1, keepdim=True)
        model.features[0][0] = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1, bias=False)
        model.features[0][0].weight.data = avg_weights
        
        # Camada final para número de classes
        num_features = model.classifier[1].in_features
        model.classifier[1] = nn.Linear(num_features, num_classes)

    elif model_name == 'efficientvit_m5':
        try:
            import timm
            model = timm.create_model('efficientvit_m5', pretrained=True, num_classes=num_classes, in_chans=1)
        except ImportError:
            print("timm não instalado. Execute: pip install timm")
            return None
        except Exception as e:
            print(f"Erro ao criar EfficientViT: {e}")
            return None
    else:
        raise ValueError(f"Modelo não suportado: {model_name}")
    
    # Contar parâmetros
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    print(f"Modelo '{model_name}' criado com sucesso")
    print(f"   Total de parâmetros: {total_params:,}")
    print(f"   Parâmetros treináveis: {trainable_params:,}")
    
    return model

print("Função de criação de modelos configurada")

Função de criação de modelos configurada


In [7]:
# =============================================================================
# FUNÇÕES DE TREINAMENTO
# =============================================================================

def train_epoch(model, train_loader, criterion, optimizer, device, scaler, monitor):
    """
    Treina o modelo por uma época.
    """
    model.train()
    running_loss = 0.0
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass com mixed precision
        with torch.cuda.amp.autocast(enabled=(device == "cuda")):
            outputs = model(images)
            loss = criterion(outputs, labels)
        
        # Backward pass
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        running_loss += loss.item()
        
        # Atualizar monitoramento de memória a cada 50 batches
        if batch_idx % 50 == 0:
            monitor.update_peak_memory()
    
    return running_loss / len(train_loader)


def validate(model, val_loader, device):
    """
    Valida o modelo.
    """
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            
            with torch.cuda.amp.autocast(enabled=(device == "cuda")):
                outputs = model(images)
            
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    return np.array(all_preds), np.array(all_labels)


def train_model(model_name, train_loader, val_loader, device, epochs, early_stopping):
    """
    Treina um modelo completo com monitoramento.
    """
    print(f"\n{'='*80}")
    print(f"TREINANDO: {model_name.upper()}")
    print(f"{'='*80}")
    
    # Inicializar monitor
    monitor = ResourceMonitor(model_name)
    monitor.start_monitoring()
    
    # Criar modelo
    model = create_model(model_name)
    if model is None:
        return None
    
    model.to(device)
    
    # Configuração de treinamento
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(model.parameters(), lr=LR)
    scaler = torch.cuda.amp.GradScaler(enabled=(device == "cuda"))
    
    # Histórico
    history = {
        'train_loss': [],
        'val_accuracy': [],
        'val_f1_macro': [],
        'val_precision': [],
        'val_recall': [],
        'combined_metric': []
    }
    
    best_combined_metric = 0.0
    best_val_f1 = 0.0
    best_val_accuracy = 0.0
    best_epoch = 0
    best_model_path = os.path.join(MODELS_PATH, f"{model_name}_best.pth")
    
    # Loop de treinamento
    for epoch in range(epochs):
        epoch_start = time.time()
        
        # Treinar
        train_loss = train_epoch(model, train_loader, criterion, optimizer, device, scaler, monitor)
        
        # Validar
        val_preds, val_labels = validate(model, val_loader, device)
        
        # Calcular métricas
        val_accuracy = accuracy_score(val_labels, val_preds)
        val_f1_macro = f1_score(val_labels, val_preds, average='macro')
        val_precision = precision_score(val_labels, val_preds, average='macro', zero_division=0)
        val_recall = recall_score(val_labels, val_preds, average='macro', zero_division=0)
        
        # Métrica combinada
        combined_metric = (F1_WEIGHT * val_f1_macro) + (ACC_WEIGHT * val_accuracy)
        
        # Salvar histórico
        history['train_loss'].append(train_loss)
        history['val_accuracy'].append(val_accuracy)
        history['val_f1_macro'].append(val_f1_macro)
        history['val_precision'].append(val_precision)
        history['val_recall'].append(val_recall)
        history['combined_metric'].append(combined_metric)
        
        epoch_time = time.time() - epoch_start
        monitor.record_epoch_time(epoch_time)
        
        # Relatório da época
        print(f"\nÉpoca [{epoch+1}/{epochs}] - Tempo: {epoch_time:.2f}s")
        print(f"   Train Loss: {train_loss:.4f}")
        print(f"   Val Acc: {val_accuracy:.4f} | Val F1: {val_f1_macro:.4f}")
        print(f"   Val Precision: {val_precision:.4f} | Val Recall: {val_recall:.4f}")
        print(f"   Métrica Combinada: {combined_metric:.4f}")
        
        # Salvar melhor modelo
        if combined_metric > best_combined_metric:
            best_combined_metric = combined_metric
            best_val_f1 = val_f1_macro
            best_val_accuracy = val_accuracy
            best_epoch = epoch
            
            # Salvar checkpoint completo em .pth
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_combined_metric': best_combined_metric,
                'best_val_f1': best_val_f1,
                'best_val_accuracy': best_val_accuracy,
                'history': history,
                'model_name': model_name
            }, best_model_path)
            
            print(f"   Modelo salvo: {best_model_path}")
        
        # Verificar early stopping
        if early_stopping(combined_metric):
            print(f"\nEarly stopping ativado na época {epoch+1}")
            break
    
    # Finalizar monitoramento
    monitoring_stats = monitor.end_monitoring()
    
    # Carregar melhor modelo
    checkpoint = torch.load(best_model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    
    return {
        'model': model,
        'model_name': model_name,
        'best_epoch': best_epoch,
        'best_val_accuracy': best_val_accuracy,
        'best_val_f1': best_val_f1,
        'best_combined_metric': best_combined_metric,
        'history': history,
        'monitoring_stats': monitoring_stats,
        'model_path': best_model_path,
        'epochs_completed': epoch + 1
    }

print("Funções de treinamento configuradas")


Funções de treinamento configuradas


In [8]:
# =============================================================================
# AVALIAÇÃO COM MATRIZ NORMALIZADA
# =============================================================================

def evaluate_model(model, test_loader, device):
    """
    Avalia modelo com métricas completas incluindo matriz de confusão normalizada.
    """
    print("\nAvaliando modelo no conjunto de teste...")
    
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Testando"):
            images, labels = images.to(device), labels.to(device)
            
            with torch.cuda.amp.autocast(enabled=(device == "cuda")):
                outputs = model(images)
            
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    # Calcular métricas
    test_accuracy = accuracy_score(all_labels, all_preds)
    test_f1_macro = f1_score(all_labels, all_preds, average='macro')
    test_f1_weighted = f1_score(all_labels, all_preds, average='weighted')
    test_precision = precision_score(all_labels, all_preds, average='macro', zero_division=0)
    test_recall = recall_score(all_labels, all_preds, average='macro', zero_division=0)
    
    # Métricas por classe
    precision_per_class, recall_per_class, f1_per_class, support = precision_recall_fscore_support(
        all_labels, all_preds, average=None, zero_division=0
    )
    
    # Matrizes de confusão
    conf_matrix = confusion_matrix(all_labels, all_preds)
    conf_matrix_normalized = confusion_matrix(all_labels, all_preds, normalize='true')
    
    # Relatório de classificação
    emotion_names = [k for k, v in sorted(EMOTION_LABELS.items(), key=lambda x: x[1])]
    class_report = classification_report(
        all_labels, all_preds,
        target_names=emotion_names,
        output_dict=True,
        zero_division=0
    )
    
    # Imprimir resultados
    print(f"\n{'='*70}")
    print("RESULTADOS DA AVALIAÇÃO - CONJUNTO DE TESTE")
    print(f"{'='*70}")
    print(f"Acurácia: {test_accuracy:.4f}")
    print(f"\nMétricas Macro:")
    print(f"   Precisão: {test_precision:.4f}")
    print(f"   Recall: {test_recall:.4f}")
    print(f"   F1-Score: {test_f1_macro:.4f}")
    print(f"\nF1-Score Weighted: {test_f1_weighted:.4f}")
    print(f"{'='*70}\n")
    
    # Imprimir métricas por classe
    print("\nMétricas por Classe:")
    print(f"{'Classe':<15} {'Precisão':<12} {'Recall':<12} {'F1-Score':<12} {'Suporte':<10}")
    print("-" * 70)
    for i, emotion in enumerate(emotion_names):
        print(f"{emotion:<15} {precision_per_class[i]:<12.4f} {recall_per_class[i]:<12.4f} "
              f"{f1_per_class[i]:<12.4f} {support[i]:<10}")
    print("-" * 70)
    
    return {
        'test_accuracy': test_accuracy,
        'test_f1_macro': test_f1_macro,
        'test_f1_weighted': test_f1_weighted,
        'test_precision': test_precision,
        'test_recall': test_recall,
        'confusion_matrix': conf_matrix,
        'confusion_matrix_normalized': conf_matrix_normalized,
        'classification_report': class_report,
        'predictions': all_preds,
        'labels': all_labels
    }

print("Função de avaliação configurada")

Função de avaliação configurada


In [9]:
def create_comprehensive_visualizations(history, conf_matrix, metrics, class_report, 
                                       experiment_id, y_true, y_pred, model_name, 
                                       train_distribution):
    """
    Visualizações completas com comparação train vs test.
    
    Args:
        train_distribution: dict com contagem de classes do treino
                           formato: {0: count, 1: count, ...}
    """
    fig = plt.figure(figsize=(24, 18))
    emotion_names = list(EMOTION_LABELS.keys())
    
    # 1. ACCURACY
    ax1 = plt.subplot(3, 4, 1)
    epochs = range(1, len(history['val_accuracy']) + 1)
    plt.plot(epochs, [acc * 100 for acc in history['val_accuracy']], 
             'b-', linewidth=2, label='Val Accuracy', marker='o', markersize=3)
    plt.title(f'{model_name.upper()}: Accuracy Evolution', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    
    # 2. LOSS
    ax2 = plt.subplot(3, 4, 2)
    plt.plot(epochs, history['train_loss'], 
             'r-', linewidth=2, label='Train Loss', marker='o', markersize=3)
    plt.title(f'{model_name.upper()}: Loss Evolution', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    
    # 3. MATRIZ RAW
    ax3 = plt.subplot(3, 4, 3)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=emotion_names, yticklabels=emotion_names, ax=ax3,
                cbar_kws={'label': 'Amostras'})
    plt.title('Matriz de Confusão (Test)', fontsize=12, fontweight='bold')
    plt.ylabel('Classe Verdadeira')
    plt.xlabel('Classe Predita')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    
    # 4. MATRIZ NORMALIZADA
    ax4 = plt.subplot(3, 4, 4)
    conf_matrix_norm = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
    conf_matrix_norm = np.nan_to_num(conf_matrix_norm)
    sns.heatmap(conf_matrix_norm, annot=True, fmt='.3f', cmap='Greens',
                xticklabels=emotion_names, yticklabels=emotion_names, ax=ax4,
                cbar_kws={'label': 'Proporção'})
    plt.title('Matriz Normalizada (Recall)', fontsize=12, fontweight='bold')
    plt.ylabel('Classe Verdadeira')
    plt.xlabel('Classe Predita')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    
    # 5. DISTRIBUIÇÃO TRAIN VS TEST ← MODIFICADO
    ax5 = plt.subplot(3, 4, 5)
    
    # Dados do TESTE
    unique_test, counts_test = np.unique(y_true, return_counts=True)
    test_dist = dict(zip(unique_test, counts_test))
    
    # Organizar dados para plotagem
    train_counts = [train_distribution.get(i, 0) for i in range(len(emotion_names))]
    test_counts = [test_dist.get(i, 0) for i in range(len(emotion_names))]
    
    x = np.arange(len(emotion_names))
    width = 0.35
    
    bars1 = plt.bar(x - width/2, train_counts, width, label='Train (Balanceado)', 
                   alpha=0.8, color='steelblue', edgecolor='black')
    bars2 = plt.bar(x + width/2, test_counts, width, label='Test (Original)', 
                   alpha=0.8, color='coral', edgecolor='black')
    
    plt.title('Distribuição: Train vs Test', fontsize=12, fontweight='bold')
    plt.ylabel('Amostras')
    plt.xlabel('Emoção')
    plt.xticks(x, emotion_names, rotation=45)
    plt.legend()
    plt.grid(True, alpha=0.3, axis='y')
    
    # Adicionar valores nas barras
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, height + max(train_counts + test_counts)*0.01,
                    f'{int(height)}', ha='center', va='bottom', fontsize=8)
    
    # 6. F1-SCORE (Test)
    ax6 = plt.subplot(3, 4, 6)
    f1_scores = [class_report[emotion]['f1-score'] for emotion in emotion_names]
    support_counts = [class_report[emotion]['support'] for emotion in emotion_names]
    bars = plt.bar(emotion_names, f1_scores, color='steelblue', alpha=0.8, edgecolor='black')
    plt.title('F1-Score por Emoção (Test)', fontsize=12, fontweight='bold')
    plt.ylabel('F1-Score')
    plt.xticks(rotation=45)
    plt.ylim(0, 1)
    for bar, score, support in zip(bars, f1_scores, support_counts):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                f'{score:.3f}\n(n={support})', ha='center', va='bottom', fontsize=9)
    
    # 7. PRECISION, RECALL, F1
    ax7 = plt.subplot(3, 4, 7)
    precision_scores = [class_report[emotion]['precision'] for emotion in emotion_names]
    recall_scores = [class_report[emotion]['recall'] for emotion in emotion_names]
    x = np.arange(len(emotion_names))
    width = 0.25
    bars1 = plt.bar(x - width, precision_scores, width, label='Precision', alpha=0.8, color='lightcoral')
    bars2 = plt.bar(x, recall_scores, width, label='Recall', alpha=0.8, color='lightblue')
    bars3 = plt.bar(x + width, f1_scores, width, label='F1-Score', alpha=0.8, color='lightgreen')
    plt.title('Métricas por Classe (Test)', fontsize=12, fontweight='bold')
    plt.ylabel('Score')
    plt.xlabel('Emoção')
    plt.xticks(x, emotion_names, rotation=45)
    plt.legend()
    plt.ylim(0, 1)
    
    # 8. F1 MACRO VS WEIGHTED
    ax8 = plt.subplot(3, 4, 8)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        y_true, y_pred, average='macro', zero_division=0)
    precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(
        y_true, y_pred, average='weighted', zero_division=0)
    metrics_comparison = {
        'Precision': [precision_macro, precision_weighted],
        'Recall': [recall_macro, recall_weighted],
        'F1-Score': [f1_macro, f1_weighted]
    }
    x = np.arange(len(metrics_comparison))
    width = 0.35
    macro_values = [metrics_comparison[metric][0] for metric in metrics_comparison]
    weighted_values = [metrics_comparison[metric][1] for metric in metrics_comparison]
    bars1 = plt.bar(x - width/2, macro_values, width, label='Macro', alpha=0.8, color='lightcoral')
    bars2 = plt.bar(x + width/2, weighted_values, width, label='Weighted', alpha=0.8, color='lightblue')
    plt.title('Macro vs Weighted (Test)', fontsize=12, fontweight='bold')
    plt.ylabel('Score')
    plt.xlabel('Métrica')
    plt.xticks(x, metrics_comparison.keys())
    plt.legend()
    plt.ylim(0, 1)
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            plt.text(bar.get_x() + bar.get_width()/2, height + 0.01,
                    f'{height:.3f}', ha='center', va='bottom', fontsize=10)
    
    # 9. HEATMAP ERROS
    ax9 = plt.subplot(3, 4, 9)
    error_matrix = conf_matrix.copy()
    np.fill_diagonal(error_matrix, 0)
    error_matrix_norm = error_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
    error_matrix_norm = np.nan_to_num(error_matrix_norm)
    sns.heatmap(error_matrix_norm, annot=True, fmt='.3f', cmap='Reds',
                xticklabels=emotion_names, yticklabels=emotion_names, ax=ax9,
                cbar_kws={'label': 'Proporção de Erros'})
    plt.title('Heatmap de Erros (Test)', fontsize=12, fontweight='bold')
    plt.ylabel('Classe Verdadeira')
    plt.xlabel('Classe Predita')
    plt.xticks(rotation=45)
    plt.yticks(rotation=0)
    
    # 10. F1 POR ÉPOCA
    ax10 = plt.subplot(3, 4, 10)
    plt.plot(epochs, history['val_f1_macro'], 'g-', linewidth=2, 
             label='Val F1-Macro', marker='o', markersize=3)
    plt.title('F1-Score Evolution', fontsize=12, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('F1-Score')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # 11. RECURSOS COMPUTACIONAIS
    ax11 = plt.subplot(3, 4, 11)
    resource_data = {
        'Tempo (min)': metrics['training_time_seconds'] / 60,
        'Memória (GB)': metrics['peak_memory_mb'] / 1024,
        'Parâmetros (M)': metrics['total_parameters'] / 1_000_000,
        'Eficiência': metrics['test_accuracy'] / (metrics['total_parameters'] / 1_000_000)
    }
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    bars = plt.bar(range(len(resource_data)), list(resource_data.values()), 
                  color=colors, alpha=0.7, edgecolor='black')
    plt.title(f'Recursos Computacionais\n{model_name.upper()}', fontsize=12, fontweight='bold')
    plt.xticks(range(len(resource_data)), resource_data.keys(), rotation=45)
    plt.ylabel('Valor')
    for bar, (key, value) in zip(bars, resource_data.items()):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + bar.get_height()*0.02,
                f'{value:.2f}', ha='center', va='bottom', fontweight='bold')
    
    # 12. RESUMO
    ax12 = plt.subplot(3, 4, 12)
    ax12.axis('off')
    
    # Calcular diferença de balanceamento
    train_max = max(train_counts)
    train_min = min(train_counts)
    test_max = max(test_counts)
    test_min = min(test_counts)
    train_ratio = train_max / train_min if train_min > 0 else 0
    test_ratio = test_max / test_min if test_min > 0 else 0
    
    summary_text = f"""
{model_name.upper()}
RESUMO

PERFORMANCE (Test):
- Accuracy: {metrics['test_accuracy']:.4f}
- F1-Macro: {f1_macro:.4f}
- F1-Weighted: {f1_weighted:.4f}

DISTRIBUIÇÃO:
- Train Ratio: {train_ratio:.1f}:1
- Test Ratio: {test_ratio:.1f}:1

RECURSOS:
- Parâmetros: {metrics['total_parameters']/1_000_000:.1f}M
- Tempo: {metrics['training_time_seconds']/60:.1f} min
- Memória: {metrics['peak_memory_mb']/1024:.2f} GB

MELHOR/PIOR (Test):
- Melhor: {emotion_names[np.argmax(f1_scores)]} ({max(f1_scores):.3f})
- Pior: {emotion_names[np.argmin(f1_scores)]} ({min(f1_scores):.3f})
    """
    ax12.text(0.05, 0.95, summary_text, fontsize=11, verticalalignment='top',
             transform=ax12.transAxes,
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
    
    plt.tight_layout()
    plt.savefig(os.path.join(PLOTS_PATH, f'{model_name}_analysis_{experiment_id}.png'), 
                dpi=300, bbox_inches='tight')
    plt.show()

In [10]:
# =============================================================================
# FUNÇÕES DE VISUALIZAÇÃO
# =============================================================================

def plot_confusion_matrices(conf_matrix, conf_matrix_normalized, model_name, save_path):
    """
    Plota matrizes de confusão (raw e normalizada).
    """
    emotion_names = [k for k, v in sorted(EMOTION_LABELS.items(), key=lambda x: x[1])]
    
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Matriz Raw
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=emotion_names, yticklabels=emotion_names, ax=axes[0],
                cbar_kws={'label': 'Amostras'})
    axes[0].set_title(f'{model_name.upper()}: Matriz de Confusão (Raw)', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Classe Verdadeira')
    axes[0].set_xlabel('Classe Predita')
    
    # Matriz Normalizada
    sns.heatmap(conf_matrix_normalized, annot=True, fmt='.3f', cmap='Greens',
                xticklabels=emotion_names, yticklabels=emotion_names, ax=axes[1],
                cbar_kws={'label': 'Proporção'})
    axes[1].set_title(f'{model_name.upper()}: Matriz Normalizada (Recall)', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Classe Verdadeira')
    axes[1].set_xlabel('Classe Predita')
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Matrizes de confusão salvas: {save_path}")

def plot_training_history(history, model_name, save_path):
    """
    Plota histórico de treinamento (Loss, Accuracy, F1).
    """
    epochs = range(1, len(history['train_loss']) + 1)
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Loss
    axes[0].plot(epochs, history['train_loss'], 'b-', linewidth=2, label='Train Loss', marker='o')
    axes[0].set_title(f'{model_name.upper()}: Loss Evolution', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Accuracy
    axes[1].plot(epochs, [acc * 100 for acc in history['val_accuracy']], 
                 'g-', linewidth=2, label='Val Accuracy', marker='o')
    axes[1].set_title(f'{model_name.upper()}: Accuracy Evolution', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # F1-Score
    axes[2].plot(epochs, history['val_f1_macro'], 'r-', linewidth=2, label='Val F1-Macro', marker='o')
    axes[2].set_title(f'{model_name.upper()}: F1-Score Evolution', fontsize=12, fontweight='bold')
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('F1-Score')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Histórico de treinamento salvo: {save_path}")


In [11]:
# =============================================================================
# SALVAR RESULTADOS
# =============================================================================

def save_results(model_name, train_result, eval_result):
    """
    Salva todos os resultados do experimento.
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Salvar métricas como JSON
    metrics = {
        'model_name': model_name,
        'timestamp': timestamp,
        'best_epoch': int(train_result['best_epoch']),
        'epochs_completed': int(train_result['epochs_completed']),
        'best_val_accuracy': float(train_result['best_val_accuracy']),
        'best_val_f1': float(train_result['best_val_f1']),
        'test_accuracy': float(eval_result['test_accuracy']),
        'test_f1_macro': float(eval_result['test_f1_macro']),
        'test_f1_weighted': float(eval_result['test_f1_weighted']),
        'test_precision': float(eval_result['test_precision']),
        'test_recall': float(eval_result['test_recall']),
        'monitoring_stats': train_result['monitoring_stats']
    }
    
    metrics_path = os.path.join(METRICS_PATH, f"{model_name}_metrics_{timestamp}.json")
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f, indent=4)
    print(f"Métricas salvas: {metrics_path}")
    
    # Salvar matrizes de confusão como CSV
    emotion_names = [k for k, v in sorted(EMOTION_LABELS.items(), key=lambda x: x[1])]
    
    conf_matrix_df = pd.DataFrame(
        eval_result['confusion_matrix'],
        index=emotion_names,
        columns=emotion_names
    )
    conf_matrix_df.to_csv(os.path.join(METRICS_PATH, f"{model_name}_confusion_raw_{timestamp}.csv"))
    
    conf_matrix_norm_df = pd.DataFrame(
        eval_result['confusion_matrix_normalized'],
        index=emotion_names,
        columns=emotion_names
    )
    conf_matrix_norm_df.to_csv(os.path.join(METRICS_PATH, f"{model_name}_confusion_normalized_{timestamp}.csv"))
    
    # Salvar visualizações
    plot_confusion_matrices(
        eval_result['confusion_matrix'],
        eval_result['confusion_matrix_normalized'],
        model_name,
        save_path=os.path.join(PLOTS_PATH, f"{model_name}_confusion_{timestamp}.png")
    )
    
    plot_training_history(
        train_result['history'],
        model_name,
        save_path=os.path.join(PLOTS_PATH, f"{model_name}_history_{timestamp}.png")
    )
    
    print(f"\nTodos os resultados salvos para {model_name}")
    
    return metrics

print("Função de salvamento configurada")

Função de salvamento configurada


In [12]:
# =============================================================================
# EXECUÇÃO PRINCIPAL
# =============================================================================

def main():
    """
    Função principal de execução.
    """
    print(f"\n{'='*80}")
    print("INICIANDO TREINAMENTO BASELINE")
    print(f"{'='*80}\n")
    
    # Carregar datasets
    print("Carregando datasets...")
    train_dataset = EmotionDataset(DATASET_PATH, split='train')

    # Calcular distribuição do conjunto de treino
    train_distribution = {}
    for _, label in train_dataset:
        label_int = int(label)
        train_distribution[label_int] = train_distribution.get(label_int, 0) + 1
    test_dataset = EmotionDataset(DATASET_PATH, split='test')
    
    # Criar data loaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
        drop_last=True
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True
    )
    
    print(f"\nDatasets carregados com sucesso")
    print(f"Train samples: {len(train_dataset)}")
    print(f"Test samples: {len(test_dataset)}")
    
    # Lista de modelos para treinar
    models_to_train = ['resnet50', 'efficientnet_b0', 'efficientvit_m5']
    
    all_results = []
    
    # Treinar cada modelo
    for model_name in models_to_train:
        try:
            # Inicializar early stopping
            early_stopping = EarlyStopping(
                patience=EARLY_STOP_PATIENCE,
                min_delta=EARLY_STOP_MIN_DELTA,
                mode='max',
                verbose=True
            )
            
            # Treinar
            train_result = train_model(
                model_name,
                train_loader,
                test_loader,
                DEVICE,
                EPOCHS,
                early_stopping
            )
            
            if train_result is None:
                continue
            
            # Avaliar
            eval_result = evaluate_model(train_result['model'], test_loader, DEVICE)
            
            # Salvar resultados
            metrics = save_results(model_name, train_result, eval_result)
            all_results.append(metrics)
            
            # Limpar memória
            del train_result, eval_result
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()
            
        except Exception as e:
            print(f"\nErro ao treinar {model_name}: {type(e).__name__}")
            print(f"Mensagem: {e}")
            import traceback
            traceback.print_exc()
            continue
    
    # Comparação final
    if all_results:
        print(f"\n{'='*80}")
        print("COMPARAÇÃO FINAL - TODOS OS MODELOS")
        print(f"{'='*80}\n")
        
        comparison_df = pd.DataFrame(all_results)
        print(comparison_df[['model_name', 'test_accuracy', 'test_f1_macro', 
                            'test_f1_weighted', 'epochs_completed']].to_string(index=False))
        
        # Salvar comparação
        comparison_path = os.path.join(METRICS_PATH, 'model_comparison.csv')
        comparison_df.to_csv(comparison_path, index=False)
        print(f"\nComparação salva: {comparison_path}")
        
        # Identificar melhor modelo
        best_model = comparison_df.loc[comparison_df['test_accuracy'].idxmax()]
        print(f"\nMelhor modelo (por acurácia): {best_model['model_name']} - {best_model['test_accuracy']:.4f}")
    
    print(f"\n{'='*80}")
    print("TREINAMENTO BASELINE CONCLUÍDO")
    print(f"{'='*80}\n")

print("Função principal configurada")


Função principal configurada


In [13]:
# =============================================================================
# EXECUTAR
# =============================================================================

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\n\nInterrompido pelo usuário")
    except Exception as e:
        print(f"\n\nERRO: {type(e).__name__}")
        print(f"Mensagem: {e}")
        import traceback
        print("\nTraceback:")
        traceback.print_exc()


INICIANDO TREINAMENTO BASELINE

Carregando datasets...

TRAIN Dataset:
   Total de imagens: 12271
   Raiva       :   705 imagens
   Nojo        :   717 imagens
   Medo        :   281 imagens
   Felicidade  :  4772 imagens
   Neutro      :  2524 imagens
   Tristeza    :  1982 imagens
   Surpresa    :  1290 imagens

TEST Dataset:
   Total de imagens: 3068
   Raiva       :   162 imagens
   Nojo        :   160 imagens
   Medo        :    74 imagens
   Felicidade  :  1185 imagens
   Neutro      :   680 imagens
   Tristeza    :   478 imagens
   Surpresa    :   329 imagens

Datasets carregados com sucesso
Train samples: 12271
Test samples: 3068

TREINANDO: RESNET50

🔍 Iniciando monitoramento: resnet50
  • Initial RAM: 730.14 MB
  • Initial GPU: 0.00 MB
Modelo 'resnet50' criado com sucesso
   Total de parâmetros: 23,516,103
   Parâmetros treináveis: 23,516,103

Época [1/100] - Tempo: 32.51s
   Train Loss: 1.0340
   Val Acc: 0.7220 | Val F1: 0.6082
   Val Precision: 0.6583 | Val Recall: 0.6042

Testando: 100%|██████████| 48/48 [00:02<00:00, 18.77it/s]



RESULTADOS DA AVALIAÇÃO - CONJUNTO DE TESTE
Acurácia: 0.7832

Métricas Macro:
   Precisão: 0.7230
   Recall: 0.6664
   F1-Score: 0.6886

F1-Score Weighted: 0.7801


Métricas por Classe:
Classe          Precisão     Recall       F1-Score     Suporte   
----------------------------------------------------------------------
Raiva           0.7402       0.5802       0.6505       162       
Nojo            0.5141       0.4562       0.4834       160       
Medo            0.6957       0.4324       0.5333       74        
Felicidade      0.8778       0.9030       0.8902       1185      
Neutro          0.7309       0.7750       0.7523       680       
Tristeza        0.7248       0.7218       0.7233       478       
Surpresa        0.7774       0.7964       0.7868       329       
----------------------------------------------------------------------
Métricas salvas: ../results/baseline/RAF-DB/metrics/resnet50_metrics_20251031_171731.json
Matrizes de confusão salvas: ../results/baseline/RAF-

Testando: 100%|██████████| 48/48 [00:01<00:00, 36.08it/s]



RESULTADOS DA AVALIAÇÃO - CONJUNTO DE TESTE
Acurácia: 0.7575

Métricas Macro:
   Precisão: 0.7031
   Recall: 0.6450
   F1-Score: 0.6661

F1-Score Weighted: 0.7555


Métricas por Classe:
Classe          Precisão     Recall       F1-Score     Suporte   
----------------------------------------------------------------------
Raiva           0.6710       0.6420       0.6562       162       
Nojo            0.4241       0.4188       0.4214       160       
Medo            0.7805       0.4324       0.5565       74        
Felicidade      0.8575       0.8886       0.8728       1185      
Neutro          0.7165       0.7397       0.7279       680       
Tristeza        0.6813       0.7155       0.6980       478       
Surpresa        0.7908       0.6778       0.7300       329       
----------------------------------------------------------------------
Métricas salvas: ../results/baseline/RAF-DB/metrics/efficientnet_b0_metrics_20251031_173133.json
Matrizes de confusão salvas: ../results/baseli

Testando: 100%|██████████| 48/48 [00:01<00:00, 38.67it/s]



RESULTADOS DA AVALIAÇÃO - CONJUNTO DE TESTE
Acurácia: 0.6969

Métricas Macro:
   Precisão: 0.6429
   Recall: 0.5712
   F1-Score: 0.5977

F1-Score Weighted: 0.6897


Métricas por Classe:
Classe          Precisão     Recall       F1-Score     Suporte   
----------------------------------------------------------------------
Raiva           0.6489       0.5247       0.5802       162       
Nojo            0.3884       0.2938       0.3345       160       
Medo            0.7561       0.4189       0.5391       74        
Felicidade      0.7930       0.8793       0.8339       1185      
Neutro          0.6434       0.6500       0.6467       680       
Tristeza        0.6293       0.5753       0.6011       478       
Surpresa        0.6409       0.6565       0.6486       329       
----------------------------------------------------------------------
Métricas salvas: ../results/baseline/RAF-DB/metrics/efficientvit_m5_metrics_20251031_175124.json
Matrizes de confusão salvas: ../results/baseli